In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from os.path import join
from tqdm import tqdm
import random
from matplotlib import pyplot as plt
import math

project_root = Path('..')

preprocess_path = join(project_root, Path('data/preprocess'))
random.seed(10)

Thoughts: thresholds need to be custom to each profile. This would be a good opportunity to try to make a profile that gets more confident the more data about the author it has. The profile could model distributions of the differences between its sentences to its mean and other people's sentences to its mean. Then it can use these two distributions to determine which is more likely for incoming sentences. 

Question: Can euclidean distances from the mean or cosine similarites be treated as normal random variables?

In [2]:
# Using function words for these experiments
function_words_train = pd.read_hdf(join(preprocess_path, "bawe_train_preprocessed_function_word_counter.hdf5"))
pos_bigrams_train = pd.read_hdf(join(preprocess_path, "bawe_train_preprocessed_pos2gram_counter.hdf5"))
function_words_train = pd.concat([function_words_train, pos_bigrams_train], axis=1)

function_words_train

0    1    2    3    4    5    6    7    8    \
author text_id sentence_position                                                
1      0       0                  1.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
               1                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
               2                  2.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
               3                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
               4                  1.0  0.0  0.0  1.0  0.0  0.0  1.0  2.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   0       110                1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               111                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               112                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               113                0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
               114                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                  9    ...  314  315  316  317  318  319  320  \
author text_id sentence_position       ...                                      
1      0       0                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               1                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               2                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               3                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               4                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   0       110                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               111                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               112                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               113                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               114                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                  321  322  323  
author text_id sentence_position                 
1      0       0                  0.0  0.0  0.0  
               1                  0.0  0.0  0.0  
               2                  0.0  0.0  0.0  
               3                  0.0  0.0  0.0  
               4                  0.0  0.0  0.0  
...                               ...  ...  ...  
6998   0       110                0.0  0.0  0.0  
               111                0.0  0.0  0.0  
               112                0.0  0.0  0.0  
               113                0.0  0.0  0.0  
               114                0.0  0.0  0.0  

[147807 rows x 433 columns]

In [3]:
def select_good_features(df):
    overall_var = df.var()

    author_vars = df.groupby(level="author").var()

    mean_explained_var = (overall_var - author_vars).mean()

    # Features that reduce the variance within classes should hopefully be good
    # features.
    selections = mean_explained_var > 0

    # The index of selctions should be the columns of the dataframe given the
    # last few operations.
    chosen_columns = selections[selections].index.tolist()

    return df[chosen_columns]

In [4]:
filtered_train = select_good_features(function_words_train)

filtered_train

0    0    7    7    12   12   13   13   32   \
author text_id sentence_position                                                
1      0       0                  1.0  1.0  2.0  2.0  0.0  0.0  0.0  0.0  0.0   
               1                  0.0  0.0  2.0  4.0  0.0  0.0  1.0  0.0  0.0   
               2                  2.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0   
               3                  0.0  0.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0   
               4                  1.0  0.0  2.0  4.0  1.0  0.0  1.0  0.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   0       110                1.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0   
               111                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               112                0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
               113                0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0   
               114                0.0  0.0  0.0  2.0  1.0  0.0  1.0  1.0  0.0   

                                  32   ...  309  310  311  312  315  317  318  \
author text_id sentence_position       ...                                      
1      0       0                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               1                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               2                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               3                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               4                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   0       110                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               111                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               112                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               113                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               114                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                  319  322  323  
author text_id sentence_position                 
1      0       0                  0.0  0.0  0.0  
               1                  0.0  0.0  0.0  
               2                  0.0  0.0  0.0  
               3                  0.0  0.0  0.0  
               4                  0.0  0.0  0.0  
...                               ...  ...  ...  
6998   0       110                0.0  0.0  0.0  
               111                0.0  0.0  0.0  
               112                0.0  0.0  0.0  
               113                0.0  0.0  0.0  
               114                0.0  0.0  0.0  

[147807 rows x 217 columns]

In [5]:
authors = filtered_train.index.get_level_values("author")
author_set = list(set(authors))

experiment_authors = random.sample(author_set, 5)

experiment_authors

[34, 6008, 6094, 16, 275]

In [6]:
chosen_author = experiment_authors[2]

chosen_author_sentences = filtered_train.loc[chosen_author]

chosen_author_sentences

0    0    7    7    12   12   13   13   32   32   \
text_id sentence_position                                                     
0       0                  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
        1                  1.0  0.0  1.0  1.0  2.0  0.0  2.0  1.0  0.0  0.0   
        2                  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3                  0.0  0.0  3.0  5.0  0.0  0.0  0.0  2.0  0.0  0.0   
        4                  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       62                 2.0  0.0  1.0  2.0  1.0  0.0  0.0  1.0  0.0  0.0   
        63                 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
        64                 1.0  0.0  1.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0   
        65                 0.0  0.0  1.0  2.0  0.0  1.0  0.0  0.0  0.0  0.0   
        66                 0.0  1.0  1.0  5.0  1.0  0.0  0.0  0.0  0.0  0.0   

                           ...  309  310  311  312  315  317  318  319  322  \
text_id sentence_position  ...                                                
0       0                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       62                 ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        63                 ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        64                 ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        65                 ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        66                 ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           323  
text_id sentence_position       
0       0                  0.0  
        1                  0.0  
        2                  0.0  
        3                  0.0  
        4                  0.0  
...                        ...  
2       62                 0.0  
        63                 0.0  
        64                 0.0  
        65                 0.0  
        66                 0.0  

[210 rows x 217 columns]

Question: If I take the mean of each sentence distance and include it in the mean instead of excluding it, does this affect the mean/var distance?

In [7]:
included_sentences_distances = np.linalg.norm(chosen_author_sentences - chosen_author_sentences.mean(), axis=1)

included_sentences_distances_mean = included_sentences_distances.mean()
included_sentences_distances_var = included_sentences_distances.var()


def get_excluded_sentence_distances(array):
    def sentence_distance(i):
        selection = [True] * len(array)
        selection[i] = False
        return np.linalg.norm(array[i] - np.mean(array[selection]))

    return np.array([sentence_distance(index) for index in range(len(array))])


excluded_sentence_distances = get_excluded_sentence_distances(chosen_author_sentences.to_numpy())

excluded_sentence_distances_mean = excluded_sentence_distances.mean()
excluded_sentence_distances_var = excluded_sentence_distances.var()

included_sentences_distances_mean, excluded_sentence_distances_mean

(5.452153443795204, 7.13358911661042)

Answer: Yes :(, the mean is consistently much lower because the sentence is included. This is unfortunate because the other method was much more efficient to compute.

In [8]:
plt.hist(excluded_sentence_distances)

plt.show()

Answer to previous question: Euclidean distances are not normal (no doy why would they be? euclidean distances from same distribution mean should on average be close to 0 and can't be lower than that).

Question: What if I just pick a threshold that accounts for 95% of the author's sentences.

In [9]:
chosen_text = chosen_author_sentences.loc[0]

chosen_sentence_distances = get_excluded_sentence_distances(chosen_text.to_numpy())

index_threshold = math.floor(len(chosen_sentence_distances) * 0.6)
threshold = np.sort(chosen_sentence_distances)[index_threshold]

threshold, index_threshold

(7.5219284044304855, 46)

In [10]:
def euclidean_distance(mean, df):
    return np.linalg.norm(mean - df, axis=1)


chosen_text_mean = chosen_text.mean()

same_texts = chosen_author_sentences.drop(index=(0,))
other_author_texts = filtered_train.drop(index=(chosen_author,))

same_sentence_classifications = pd.DataFrame(
    euclidean_distance(chosen_text_mean, same_texts) > threshold, index=same_texts.index
)
other_sentence_classifications = pd.DataFrame(
    euclidean_distance(chosen_text_mean, other_author_texts) > threshold, index=other_author_texts.index
)

same_text_classifications = same_sentence_classifications.groupby(level=("text_id")).mean() > 0.5
other_text_classifications = other_sentence_classifications.groupby(level=("author", "text_id")).mean() > 0.5

same_flags = same_text_classifications.sum()
same_length = len(same_text_classifications)
other_flags = other_text_classifications.sum()
other_length = len(other_text_classifications)

tnr = (same_length - same_flags) / same_length
tpr = other_flags / other_length

tnr[0], tpr[0]

(1.0, 0.0015564202334630351)